In [1]:
import json
import requests


class UmlsApi:
    granting_ticket = ""
    username = ""
    password = ""

    def __init__(self, username, password):
        self.username = username;
        self.password = password;
        self.setTgT()

    def setTgT(self):
        data = {'username': self.username, 'password': self.password}
        r = requests.post('https://utslogin.nlm.nih.gov/cas/v1/tickets', data)
        self.granting_ticket = r.headers['location'].rsplit('/', 1)[-1]
        # print(self.granting_ticket)

    def getSingleTicket(self):
        data = {'service': 'http://umlsks.nlm.nih.gov'}
        url = "https://utslogin.nlm.nih.gov/cas/v1/api-key/{0}".format(self.granting_ticket)
        # print(url)
        r = requests.post(url, data)
        return r.text

    def cuiSearch(self, cui):
        ticket = self.getSingleTicket()
        url = "https://uts-ws.nlm.nih.gov/rest/content/current/CUI/{1}?ticket={0}".format(ticket, cui)
        #print(url)
        r = requests.get(url)
        # print r.text
        return r.text

    def getRelatedCuis(self, cui, cuis):
        cui_search_response_string = self.cuiSearch(cui)
        cui_search_response_json = json.loads(cui_search_response_string)
        cuis[cui] = cui_search_response_json['result']['name']
        #print("SEARCH", cui_search_response_json)

        ticket = self.getSingleTicket()
        relation_url = cui_search_response_json['result']['relations'] + "?/pageSize=1000&ticket={0}".format(ticket)
        # print(relation_url)
        relation_response_str = requests.get(relation_url).text
        relation_response_json = json.loads(relation_response_str)
        #print("RELATION", relation_response_json)

        for conceptRelation in relation_response_json['result']:
            if conceptRelation['relationLabel'] == 'RB':
                relatedCui = conceptRelation['relatedId'].rsplit('/', 1)[-1]
                self.getRelatedCuis(relatedCui, cuis)


In [2]:
umlsapi = UmlsApi("harryhoch", "pudwallF00bar45")

In [8]:
cuis = {}
umlsapi.getRelatedCuis("C0015967", cuis)

In [9]:
cuis

{'C0015967': 'Fever',
 'C0009952': 'Febrile Convulsions',
 'C1852577': 'FEBRILE CONVULSIONS, FAMILIAL, 1 (disorder)',
 'C1858493': 'FEBRILE CONVULSIONS, FAMILIAL, 4',
 'C1865342': 'FEBRILE CONVULSIONS, FAMILIAL, 2 (disorder)',
 'C0030612': 'Animal Milk Fever',
 'C0031069': 'Familial Mediterranean Fever',
 'C0235839': 'Fever of the newborn',
 'C0239572': 'extreme fever',
 'C0239573': 'Fever, high-grade',
 'C0239574': 'Low grade fever',
 'C2199828': 'low-grade fever (99-101 degrees Fahrenheit)',
 'C0856281': 'Pyrexia of unknown origin (excl puerperal)',
 'C0864638': 'Cerebrospinal fever',
 'C1320835': 'Drug-induced hyperpyrexia',
 'C1856788': 'Fever, Familial Lifelong Persistent'}

In [4]:
import pandas as pd

In [5]:
ontfile= pd.read_excel('ontology_code_v2-hh-mismatch.xlsx')

In [6]:
ontfile

,Category 1,Cateogy 2,Finding or Comorbidity,UMLS code,Meaning in UTS query,Extracted via MetaMapLite (if not as in column D),Unnamed: 6,Unnamed: 7,prior
0,finding in NEJM,finding in Topaz finding set,fever,C0015967,"higher than 38 C, 100.4 F",NaN,NaN,NaN,NaN
1,finding in NEJM,finding in Topaz finding set,Coughing,C0010200,dry cough,"C1961131 (Cough, CTCAE), C4084727 (Cough Frequ...",NaN,NaN,C0850149
2,finding in NEJM,finding in Topaz finding set,nausea,C0027497,NaN,NaN,NaN,NaN,NaN
3,finding in NEJM,finding in Topaz finding set,diarrhea,C0011991,NaN,NaN,NaN,NaN,NaN
4,finding in NEJM,finding in Topaz finding set,nasal congestion,C0027424,NaN,NaN,NaN,NaN,NaN
5,finding in NEJM,finding in Topaz finding set,headache,C0018681,NaN,NaN,NaN,NaN,NaN
6,finding in NEJM,finding in Topaz finding set,sore throat,C0242429,NaN,NaN,NaN,NaN,NaN
7,finding in NEJM,finding in Topaz finding set,fatigue,C0015672,NaN,NaN,NaN,NaN,NaN
8,finding in NEJM,finding in Topaz finding set,hemoptysis,C0019079,NaN,NaN,NaN,NaN,NaN
9,finding in NEJM,finding in Topaz finding set,shortness of breath,C0013404,dyspnea,NaN,NaN,NaN,NaN


In [7]:
codes = ontfile['UMLS code'].values.tolist()

set the UMLs user name and password. 

In [ ]:
username=
password=

In [8]:
umlsapi = UmlsApi(username, password)
masterCuiList={}
for code in codes:
    cuis = {}
    umlsapi.getRelatedCuis(code, cuis)
    masterCuiList.update(cuis)

In [9]:
len(masterCuiList.keys())

1087

In [10]:
import jsonpickle

In [11]:
cuistring = jsonpickle.encode(masterCuiList)
f =open("fullcuilist.json",'w')
f.write(cuistring)
f.close()